# Praktikum 2
## Feed Forward Neural Network


___Alvin Sullivan 13515048___

___Albertus Djauhari Djohan 13515054___

___Kevin 13515138___

### Multi Layer Neural Network

#### Implementasi Algoritma Backpropagation



In [52]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.layers import Dropout
import keras

In [53]:
var_nodes = [3, 2, 1]
var_epoch = 10
var_momentum = 0.001
var_learning_rate = 0.5

#### Eksekusi Dataset

- Membaca dataset
- Praproses Data (Continuous dan Kategorikal)
    - Kategorikal menggunakan StandardScaler

In [54]:
file = "dataset/data_training_praktikum.csv"
data = pd.read_csv(file)

#Handle Categorical Data
le = preprocessing.LabelEncoder()
data['loc'] = le.fit_transform(data['loc'])
data['struct1'] = le.fit_transform(data['struct1'])
data['struct2'] = le.fit_transform(data['struct2'])
data['struct3'] = le.fit_transform(data['struct3'])
data['cit1'] = le.fit_transform(data['cit1'])
data['cit2'] = le.fit_transform(data['cit2'])
data['labelclass'] = le.fit_transform(data['labelclass'])
data = data.values
X = data[:, 0:-1]
y = data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    random_state=43)

print('X_train')
print(X_train)

print('X_test')
print(X_test)

print('y_train')
print(y_train)

print('y_test')
print(y_test)

X_train
[[1 0 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
X_test
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_train
[ 8 10 15 ...  7  6 10]
y_test
[10  8  5 ... 10  8 10]


In [55]:
class MultiLayerNN:

    def __init__(self, data, hidden_node, output_node,
        num_batch, learning_rate_const, tolerance_const,
        **kwargs):
        
        self.instance = data
        self.w_hidden_node = hidden_node
        self.w_output_node = output_node
        # Gradient Descent Parameters
        self.batch_size = num_batch
        self.learning_rate = learning_rate_const
        self.tolerance = tolerance_const
        self.momentum = kwargs.get('momentum', 0)
        self.epochs = kwargs.get('epochs', 10)
       
    def train(self, instance_target):
        instance_target_t = np.array([instance_target]).T
        batch_iteration = int(np.ceil(self.instance.shape[0] / self.batch_size))
        old_loss = -np.inf
        for step in range(self.epochs):
            loss = 0
            for i in range(batch_iteration):
                start_index = i * self.batch_size
                end_index = i * self.batch_size + self.batch_size
                if end_index > len(instance_target_t):
                    end_index = len(instance_target_t)
                o_out = self.gradient_descent(self.instance[start_index:end_index:1], 
                                              instance_target_t[start_index:end_index:1])
                loss = loss + self.loss_function(o_out, 
                                            instance_target_t[start_index:end_index:1])
        # Print Loss
            print ("Loss after epoch %i: %f" % (step, loss/self.instance.shape[0]))
            
            if np.abs(loss - old_loss) < self.tolerance:
                break
            old_loss = loss
    
    def feed_forward(self, instance):
        s = list()
        o = list()
        # Feed Forward Hidden Node
        s_temp = instance.dot(self.w_hidden_node[0].T)
        o_temp = self.sigmoid(s_temp)
        s.append(s_temp)
        o.append(o_temp)
        iteration = len(self.w_hidden_node)
        for i in range(1,iteration):
            s_temp = o[i-1].dot(self.w_hidden_node[i].T)
            o_temp = self.sigmoid(s_temp)
            o.append(o_temp)
        # Feed Forward Output Node
        s_out = o[-1].dot(self.w_output_node.T)
        o_out = self.sigmoid(s_out)
        return s, o, s_out, o_out

    def sigmoid(self, X):
        output = 1 / (1 + np.exp(-X))
        return np.matrix(output)

    def loss_function(self, o_out, instance_target):
        
        squared_error = np.square(instance_target - o_out)
        data_loss = np.sum(squared_error)      
        return data_loss    
    def back_propagation(self, instance_target, o, o_out):
        d = list()
        # Back Propagation Output Node
        d_temp = np.multiply(np.multiply(o_out, 1-o_out), instance_target-o_out)
        d.insert(0, d_temp)
        # Back Propagation Hidden Node
        d_temp = np.multiply(np.multiply(o[-1], 1-o[-1]), 
                             (self.w_output_node.T.dot(d[0].T)).T)
        d.insert(0, d_temp)
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            d_temp = np.multiply(np.multiply(o[i-1], 1-o[i-1]), 
                                 (self.w_hidden_node[i].T.dot(d[0].T)).T)
            d.insert(0, d_temp)
        return d

    def update_weight(self, instance, o, d):
        # Update Weight Output Node
        self.w_output_node[0] = self.w_output_node[0] \
        + self.momentum * self.w_output_node[0] \
        + self.learning_rate * d[-1].T.dot(o[-1])
        # Update Weight Hidden Node
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            self.w_hidden_node[i] = self.w_hidden_node[i] \
            + self.momentum * self.w_hidden_node[i] \
            + self.learning_rate * d[i].T.dot(o[i-1])
        self.w_hidden_node[0] = self.w_hidden_node[0] \
        + self.momentum * self.w_hidden_node[0] \
        + self.learning_rate * d[0].T.dot(instance)
    def gradient_descent(self, instance, instance_target):
        # Feed Forward
        _,o,_,o_out = self.feed_forward(instance)
        # Back Propagation      
        d = self.back_propagation(instance_target, o, o_out)
        # Update Weight
        self.update_weight(instance, o, d)
        return o_out
    def predict(self, instance):
        _,_,s_out,o_out = self.feed_forward(instance)
        return o_out

#### Implementasi Keras



In [56]:
model = Sequential()

for i in range(0,len(var_nodes)):
    if i == 0:
        model.add(Dense(units=var_nodes[i], activation='sigmoid', 
                        input_dim=X_train.shape[1]))
    else:
        model.add(Dense(units=var_nodes[i], activation='sigmoid'))
        
sgd = optimizers.SGD(lr=var_learning_rate, decay=0, 
                     momentum=var_momentum, nesterov=False)

model.compile(optimizer=sgd,
              loss='mean_squared_error',
              metrics=['accuracy'])

### Perbandingan Hasil Algoritma Backpropagation dan Keras

#### Inisialisasi bobot awal hidden node dan output node

In [57]:
np.random.seed(0)
w_hidden_node = list()
for i in range(0,len(var_nodes)-1):
    if i == 0:
        w_temp = np.random.randn(var_nodes[i],
                                 X_train.shape[1]) / np.sqrt(var_nodes[i])
    else:
        w_temp = np.random.randn(var_nodes[i], 
                                 var_nodes[i-1]) / np.sqrt(var_nodes[i])
    w_hidden_node.append(w_temp)

w_output_node = np.random.randn(var_nodes[-1], var_nodes[-2])

print('Matriks weight hidden node:\n', w_hidden_node)
print('Matriks weight output node:\n', w_output_node)

Matriks weight hidden node:
 [array([[ 1.0184761 ,  0.23103087,  0.56507464,  1.29378029,  1.07823511,
        -0.56423165,  0.5485338 , -0.08738612, -0.05959343,  0.23705916,
         0.08316359,  0.8396252 ,  0.43938534,  0.0702491 ,  0.25626456,
         0.19264696,  0.86260696, -0.11844818,  0.18074972, -0.4931124 ,
        -1.47396936,  0.37736687,  0.49908247, -0.42848917,  1.31044344,
        -0.83967841,  0.02641869, -0.10807065,  0.88495049,  0.84833468,
         0.08945894,  0.21833223, -0.51256334, -1.14361337, -0.20086717,
         0.09026812,  0.71030866,  0.69419433, -0.22362324, -0.17453457,
        -0.60538234, -0.81984774, -0.98511555,  1.1262807 , -0.29424782,
        -0.25292232, -0.72330174,  0.44888427, -0.93178436, -0.12282566,
        -0.51699786,  0.22337826, -0.29491348, -0.68163831, -0.01627102,
         0.24729752,  0.03840374,  0.17463223, -0.36622603],
       [-0.20942871, -0.38824522, -0.20758811, -0.46947022, -0.99666973,
         0.10243703, -0.23196833,

#### Implementasi Mini-Batch (Batch_size = 1)

In [58]:
var_batch = 1

##### Classifier Backpropagation

In [59]:
multiLayerNN = MultiLayerNN(X_train, w_hidden_node, w_output_node, var_batch, 
                            var_learning_rate, 1e-6, 
                            momentum = var_momentum, epochs = var_epoch)
multiLayerNN.train(y_train)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

y_test_res = multiLayerNN.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(y_test_res)

/home/sulley/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: overflow encountered in exp


Loss after epoch 0: 78.083254
Loss after epoch 1: 78.078503
Loss after epoch 2: 78.078503
Matriks weight hidden node:
 [matrix([[ 5.85450726e+14,  1.39303178e+14,  3.24848413e+14,
          7.43780363e+14,  6.28107856e+14, -3.10157264e+14,
          3.15980917e+14, -4.96233090e+13, -2.65344465e+13,
          1.59414248e+14,  5.01266697e+13,  4.83689182e+14,
          2.52550371e+14,  3.99248125e+13,  1.47296015e+14,
          1.10729826e+14,  4.95811616e+14, -6.80817702e+13,
          1.03432401e+14, -2.83431673e+14, -8.47666012e+14,
          2.16893522e+14,  2.86860489e+14, -2.46258155e+14,
          7.53218076e+14, -4.82631066e+14,  1.51849641e+13,
         -6.21169610e+13,  5.08654938e+14,  4.87606711e+14,
          5.14198092e+13,  1.25493250e+14, -2.94640389e+14,
         -6.57327312e+14, -1.15454647e+14,  5.18844057e+13,
          4.08271965e+14,  3.99009755e+14, -1.28534405e+14,
         -1.00319168e+14, -3.47962303e+14, -4.71232955e+14,
         -5.66225766e+14,  6.47364819e+1

##### Classifier menggunakan keras

In [60]:
model.fit(X_train, y_train, batch_size=var_batch, epochs=var_epoch, verbose=1)
prediction = model.predict(X_test)
score = model.evaluate(X_test, y_test, verbose=0)
print("Hasil Prediksi Train Test: ")
print(prediction)
print("Hasil Pengukuran Perfomansi Train Test: ")
print(score)

Epoch 1/10
11334/11334 [==============================] - 9s 754us/step - loss: 78.0757 - acc: 0.0087
Epoch 2/10
11334/11334 [==============================] - 9s 820us/step - loss: 78.0750 - acc: 0.0087
Epoch 3/10
11334/11334 [==============================] - 11s 933us/step - loss: 78.0750 - acc: 0.0087
Epoch 4/10
11334/11334 [==============================] - 15s 1ms/step - loss: 78.0750 - acc: 0.0087
Epoch 5/10
11334/11334 [==============================] - 11s 1ms/step - loss: 78.0750 - acc: 0.0087
Epoch 6/10
11334/11334 [==============================] - 14s 1ms/step - loss: 78.0750 - acc: 0.0087
Epoch 7/10
11334/11334 [==============================] - 19s 2ms/step - loss: 78.0750 - acc: 0.0087
Epoch 8/10
11334/11334 [==============================] - 14s 1ms/step - loss: 78.0750 - acc: 0.0087
Epoch 9/10
11334/11334 [==============================] - 14s 1ms/step - loss: 78.0750 - acc: 0.0087
Epoch 10/10
11334/11334 [==============================] - 14s 1ms/step - loss: 78.0750

#### Implementasi Mini-Batch (Batch_size = jumlah_data)¶

In [61]:
var_batch = X_train.shape[0]

##### Classifier Backpropagation

In [62]:
multiLayerNN = MultiLayerNN(X_train, w_hidden_node, w_output_node, var_batch, 
                            var_learning_rate, 1e-6, 
                            momentum = var_momentum, epochs = var_epoch)
multiLayerNN.train(y_train)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

y_test_res = multiLayerNN.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(y_test_res)

Loss after epoch 0: 78.078503
Loss after epoch 1: 78.078503
Matriks weight hidden node:
 [matrix([[ 5.86622213e+14,  1.39581923e+14,  3.25498434e+14,
          7.45268668e+14,  6.29364700e+14, -3.10777889e+14,
          3.16613195e+14, -4.97226052e+13, -2.65875419e+13,
          1.59733236e+14,  5.02269731e+13,  4.84657044e+14,
          2.53055724e+14,  4.00047021e+13,  1.47590754e+14,
          1.10951397e+14,  4.96803735e+14, -6.82180018e+13,
          1.03639370e+14, -2.83998819e+14, -8.49362192e+14,
          2.17327526e+14,  2.87434497e+14, -2.46750917e+14,
          7.54725266e+14, -4.83596811e+14,  1.52153492e+13,
         -6.22412570e+13,  5.09672756e+14,  4.88582412e+14,
          5.15227003e+13,  1.25744362e+14, -2.95229965e+14,
         -6.58642624e+14, -1.15685672e+14,  5.19882264e+13,
          4.09088917e+14,  3.99808174e+14, -1.28791602e+14,
         -1.00519907e+14, -3.48658576e+14, -4.72175892e+14,
         -5.67358784e+14,  6.48660196e+14, -1.69466503e+14,
         -

/home/sulley/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: overflow encountered in exp


##### Classifier menggunakan keras

In [63]:
model.fit(X_train, y_train, batch_size=var_batch, epochs=var_epoch, verbose=1)
score = model.evaluate(X_test, y_test, verbose=0)
prediction = model.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(prediction)
print("Hasil Pengukuran Perfomansi Train Test: ")
print(score)

Epoch 1/10
11334/11334 [==============================] - 0s 11us/step - loss: 78.0750 - acc: 0.0087
Epoch 2/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 3/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 4/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 5/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 6/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 7/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 8/10
11334/11334 [==============================] - 0s 1us/step - loss: 78.0750 - acc: 0.0087
Epoch 9/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.0087
Epoch 10/10
11334/11334 [==============================] - 0s 2us/step - loss: 78.0750 - acc: 0.008

#### Perbandingan & Analisis Hasil Classifier A dan B



### Pembagian Tugas
1. Alvin Sullivan - 13515048 - 
2. Albertus Djauhari - 13515054 - 
3. Kevin - 13515138 - 